In [1]:
# import the necessary packages
import numpy as np
import cv2
import os

In [2]:
#load the files and directories
face = os.path.join("face_detector")
age = os.path.join("age_detector")


In [3]:
# define the list of age buckets our age detector will predict
AGE_BUCKETS = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)","(38-43)", "(48-53)", "(60-100)"]

In [4]:
# load our serialized face detector model from disk
prototxtPath = os.path.sep.join([face, "deploy.prototxt"])
weightsPath = os.path.sep.join([face,
	"res10_300x300_ssd_iter_140000.caffemodel"])
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [5]:
# load our serialized age detector model from disk
prototxtPath = os.path.sep.join([age, "age_deploy.prototxt"])
weightsPath = os.path.sep.join([age, "age_net.caffemodel"])
ageNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [12]:
# load the input image and construct an input blob for the image
image = cv2.imread("images/neil_patrick_harris.png")
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(image, 1.0, (300,300), (104.0, 177.0, 123.0))

In [13]:
# pass the blob through the network and obtain the face detections
faceNet.setInput(blob)
detections = faceNet.forward()

In [14]:
# loop over the detections
conf=0.5
for i in range(0, detections.shape[2]):
     #extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0,0,i,2]
        # filter out weak detections by ensuring the confidence is greater than the minimum confidence
        if confidence>conf:
            # compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0,0,i,3:7] * np.array([w,h,w,h])
            (startX,startY,endX,endY) = box.astype("int")
            
            # extract the ROI of the face and then construct a blob from *only* the face ROI
            f = image[startY:endY, startX:endX]
            faceblob = cv2.dnn.blobFromImage(f , 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
            
            ageNet.setInput(faceblob)
            preds = ageNet.forward()
            i = preds[0].argmax()
            a = AGE_BUCKETS[i]
            ageConfidence = preds[0][i]
            
            # display the predicted age to our terminal
            text = "{}: {:.2f}%".format(a, ageConfidence * 100)
            print("[INFO] {}".format(text))
            
            # draw the bounding box of the face along with the associated# predicted age
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(image, (startX, startY), (endX, endY),
                          (0, 0, 255), 2)
            cv2.putText(image, text, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            
# display the output image
cv2.imshow("Image", image)
cv2.waitKey(0)            

[INFO] (8-12): 54.28%


27